In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm  # 导入 tqdm 库

# 设置随机种子以确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 检查设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'使用设备: {device}')

# 1. 数据预处理与加载
def get_dataloaders(batch_size=128, num_workers=2):
    # 数据增强与标准化
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),  # 随机裁剪
        transforms.RandomHorizontalFlip(),     # 随机水平翻转
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), 
                             (0.2675, 0.2565, 0.2761)),  # CIFAR-100 的均值和标准差
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), 
                             (0.2675, 0.2565, 0.2761)),
    ])

    # 加载训练集
    trainset = torchvision.datasets.CIFAR100(
        root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    # 加载验证集
    testset = torchvision.datasets.CIFAR100(
        root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=100, shuffle=False, num_workers=num_workers)

    return trainloader, testloader

# 2. 模型定义与调整
def get_model(num_classes=100, pretrained=True):
    # 加载预训练的 ResNet50 模型
    model = models.resnet50(pretrained=pretrained)

    if pretrained:
        # 修改第一层卷积层以适应 CIFAR-100 的输入（32x32）
        # 原始 ResNet50 的 conv1: kernel_size=7, stride=2, padding=3
        # 调整为 kernel_size=3, stride=1, padding=1
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # 移除最大池化层
        model.maxpool = nn.Identity()
    else:
        # 如果不使用预训练权重，保持之前的修改
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()

    # 修改最后的全连接层以适应 CIFAR-100
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    return model.to(device)

# 3. 定义损失函数与优化器
class LabelSmoothingCrossEntropy(nn.Module):
    """
    标签平滑交叉熵损失函数
    """
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, x, target):
        log_probs = nn.functional.log_softmax(x, dim=-1)
        # 创建平滑后的标签
        targets = torch.zeros_like(x).scatter_(1, target.unsqueeze(1), 1)
        targets = targets * self.confidence + self.smoothing / x.size(1)
        loss = (-targets * log_probs).sum(dim=1).mean()
        return loss

# 4. 训练与验证函数
def train_one_epoch(model, criterion, optimizer, dataloader, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f'训练 Epoch {epoch}')
    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # 更新进度条描述
        progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f'训练集 - Epoch: {epoch} | Loss: {epoch_loss:.4f} | 准确率: {epoch_acc:.2f}%')
    return epoch_loss, epoch_acc

def validate(model, criterion, dataloader, epoch, print_softmax=True, samples_to_print=5):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    softmax_distributions = []
    all_softmax = []
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f'验证 Epoch {epoch}')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # 计算 Softmax 分布
            softmax = nn.functional.softmax(outputs, dim=1)
            softmax_distributions.append(softmax.cpu().numpy())
            all_softmax.append(softmax.cpu())

            # 更新进度条描述
            progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f'验证集 - Epoch: {epoch} | Loss: {epoch_loss:.4f} | 准确率: {epoch_acc:.2f}%')

    # 计算 Softmax 分散情况（标准差）
    softmax_distributions = np.concatenate(softmax_distributions, axis=0)
    std_softmax = np.std(softmax_distributions, axis=0)
    mean_std = np.mean(std_softmax)
    print(f'Softmax 平均标准差: {mean_std:.4f}')

    # 打印部分 Softmax 概率分布
    if print_softmax:
        all_softmax = torch.cat(all_softmax, dim=0)  # (N, C)
        print(f'\n--- 前 {samples_to_print} 个样本的 Softmax 概率分布 ---')
        for i in range(min(samples_to_print, all_softmax.size(0))):
            probs = all_softmax[i].cpu().numpy()
            top3_idx = probs.argsort()[-3:][::-1]
            top3_probs = probs[top3_idx]
            top3_classes = [f'{idx}' for idx in top3_idx]
            print(f'样本 {i+1}:')
            for cls, prob in zip(top3_classes, top3_probs):
                print(f'  类别 {cls}: {prob:.4f}')
            print('')

    return epoch_loss, epoch_acc, mean_std, softmax_distributions

# 5. 绘制损失与准确率曲线
def plot_metrics(epochs, train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(12,5))

    # 损失曲线
    plt.subplot(1,2,1)
    plt.plot(epochs, train_losses, label='训练损失')
    plt.plot(epochs, val_losses, label='验证损失')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('训练与验证损失')
    plt.legend()

    # 准确率曲线
    plt.subplot(1,2,2)
    plt.plot(epochs, train_accuracies, label='训练准确率')
    plt.plot(epochs, val_accuracies, label='验证准确率')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('训练与验证准确率')
    plt.legend()

    plt.tight_layout()
    plt.show()

# 6. 主训练过程
def main():
    # 超参数
    num_epochs = 100
    batch_size = 128
    learning_rate = 0.1
    smoothing = 0.1
    use_pretrained = True  # 是否使用预训练权重

    # 创建数据加载器
    trainloader, testloader = get_dataloaders(batch_size=batch_size)

    # 初始化模型
    model = get_model(num_classes=100, pretrained=use_pretrained)

    # 定义损失函数与优化器
    criterion = LabelSmoothingCrossEntropy(smoothing=smoothing)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    # 存储训练过程中的指标
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    softmax_stds = []

    # 创建保存模型的目录
    os.makedirs('checkpoints', exist_ok=True)

    for epoch in range(1, num_epochs + 1):
        print(f'\n--- Epoch {epoch}/{num_epochs} ---')
        train_loss, train_acc = train_one_epoch(model, criterion, optimizer, trainloader, epoch)
        val_loss, val_acc, softmax_std, softmax_distributions = validate(model, criterion, testloader, epoch)

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        softmax_stds.append(softmax_std)

        scheduler.step()

        # 每10个epoch保存一次模型
        if epoch % 10 == 0:
            checkpoint_path = f'checkpoints/resnet50_cifar100_epoch{epoch}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f'已保存模型到 {checkpoint_path}')

    # 绘制训练与验证的损失与准确率
    epochs_range = range(1, num_epochs + 1)
    plot_metrics(epochs_range, train_losses, val_losses, train_accuracies, val_accuracies)

    # 打印详细的 Softmax 分散情况
    print('\n--- Softmax 分散情况 ---')
    std_per_class = np.std(softmax_distributions, axis=0)
    for i, std in enumerate(std_per_class[:10]):  # 仅显示前10个类别，避免过长输出
        print(f'类别 {i:3d} 的 Softmax 标准差: {std:.4f}')
    print(f'所有类别的平均 Softmax 标准差: {np.mean(std_per_class):.4f}')

if __name__ == '__main__':
    main()


使用设备: cuda
Files already downloaded and verified
Files already downloaded and verified

--- Epoch 1/100 ---


训练 Epoch 1: 100%|██████████| 391/391 [01:32<00:00,  4.22it/s, Loss=2.8819]


训练集 - Epoch: 1 | Loss: 3.5082 | 准确率: 21.46%


验证 Epoch 1: 100%|██████████| 100/100 [00:06<00:00, 14.79it/s, Loss=3.8754]


验证集 - Epoch: 1 | Loss: 4.8640 | 准确率: 24.04%
Softmax 平均标准差: 0.0314

--- 前 5 个样本的 Softmax 概率分布 ---
样本 1:
  类别 39: 0.1487
  类别 68: 0.1362
  类别 54: 0.0574

样本 2:
  类别 33: 0.1370
  类别 65: 0.1042
  类别 96: 0.0533

样本 3:
  类别 30: 0.1335
  类别 55: 0.0767
  类别 54: 0.0732

样本 4:
  类别 65: 0.0756
  类别 37: 0.0662
  类别 54: 0.0618

样本 5:
  类别 71: 0.6843
  类别 23: 0.2059
  类别 60: 0.0709


--- Epoch 2/100 ---


...

训练 Epoch 35: 100%|██████████| 391/391 [01:27<00:00,  4.48it/s, Loss=1.0765]


训练集 - Epoch: 35 | Loss: 1.1077 | 准确率: 89.97%


验证 Epoch 35: 100%|██████████| 100/100 [00:06<00:00, 14.86it/s, Loss=1.3345]


验证集 - Epoch: 35 | Loss: 1.4727 | 准确率: 77.69%
Softmax 平均标准差: 0.0786

--- 前 5 个样本的 Softmax 概率分布 ---
样本 1:
  类别 49: 0.1391
  类别 68: 0.1172
  类别 76: 0.1124

样本 2:
  类别 33: 0.9225
  类别 96: 0.0208
  类别 84: 0.0064

样本 3:
  类别 72: 0.5175
  类别 55: 0.2772
  类别 4: 0.1372

样本 4:
  类别 51: 0.9162
  类别 91: 0.0046
  类别 37: 0.0045

样本 5:
  类别 71: 0.7923
  类别 23: 0.1081
  类别 60: 0.0262


--- Epoch 36/100 ---


训练 Epoch 36: 100%|█████████▉| 390/391 [01:30<00:00,  4.19it/s, Loss=1.0126]